In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini text - Single turn

## Set environment

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [8]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [10]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [ ]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash" # @param {type:"string"}

### Import libraries

In [11]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Part,
)

### Initalize Vertex AI

In [ ]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function

In [12]:
def generate(query:str,
             stream:bool) -> GenerationResponse:

    """
    Generate a response from the model.

    Args:
      query : query string to generate response
      stream : whether to stream the response

    Returns:
      GenerationResponse: The generated response.

    """

    # Request body
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses


### Execution

In [13]:
from time import perf_counter

t1_start = perf_counter()

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        다음 지침에서 AI를 사용하는 것의 장단점을 설명하세요.

        1. 생성 AI가 무엇인지 정의하세요.
        2. 일반적인 머신 러닝과 생성 AI의 차이점은 무엇인가요?
        3. 생성 AI를 사용하는 것의 장단점은 무엇인가요?
        4. 주요 산업에서 생성 AI의 사용 사례를 설명하세요.
"""

stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")


Token Count : total_tokens: 131
total_billable_characters: 183


 Usage metadata:
{'prompt_token_count': 131, 'candidates_token_count': 850, 'total_token_count': 981}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.0247802734
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0903320312
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0240478516
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.137695312
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.0927734375
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0864257812
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0346679688
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0446777344
]


## 생성 AI에 대한 이해를 돕는 가이드

### 1. 생성 AI란 무엇일까요?

**생성 AI**는 텍스트, 이미지, 오디오, 비디오와 같은 새로운 콘텐츠를 생성하는 인공지능의 한 종류입니다. 기존 데이터를 학습하여 새로운 데이터를 생성하는 능력을 갖추고 있으며, 이는 예술, 엔터테인먼트, 교육, 연구 등 다양한 분야에서 혁신적인 가능성을 열어줍니다.

### 2. 일반적인 머신 러닝과 생성 AI의 차이점은 무엇인가요?

**일반적인 머신 러닝**은 주어진 데이터를 기반으로 미래의 결과를 예측하는 데 초점을 맞춥니다. 예를 들어, 이메일 스팸 필터는 이전 스팸 이메일 데이터를 학습하여 새로운 이메일이 스팸인지 아닌지 판단합니다.

반면 **생성 AI**는 새로운 콘텐츠를 생성하는 데 초점을 맞춥니다.  예를 들어, 이미지 생성 AI는 기존 이미지 데이터를 학습하여 새로운 이미지를 만들어낼 수 있습니다. 

즉, 머신 러닝은 **예측**에 중점을 두고, 생성 AI는 **창조**에 중점을 둡니다.

### 3. 생성 AI 사용의 장단점은 무엇일까요?

**장점:**

* **창의성 증진:**  새로운 아이디어와 콘텐츠를 생성하여 창의적인 작업을 촉진합니다.
* **효율성 향상:**  반복적인 작업을 자동화하여 시간과 비용을 절약합니다.
* **개인 맞춤형 경험 제공:**  개인의 취향과 요구에 맞는 맞춤형 콘텐츠를 생성합니다.
* **새로운 가능성 창출:**  기존 방식으로는 불가능했던 새로운 콘텐츠와 서비스를 제공합니다.

**단점:**

* **데이터 편향 위험:**  훈련 데이터의 편향이 생성된 콘텐츠에도 영향을 미칠 수 있습니다.
* **저작권 문제:**  생성된 콘텐츠의 저작권 및 소유권에 대한 문제 발생 가능성이 있습니다.
* **오용 가능성:**  가짜 뉴스나 악의적인 콘텐츠를 생성하는 데 악용될 수 있습니다.
* **윤리적 딜레마:**  생성 AI의 윤리적 책임 및 사용에 대한 논쟁이 발생할 수 있습니다.

### 4. 주요 산업에서 생성 AI의 사용 사례

* **예술 및 엔터테인먼트:**  새로운 음악, 그림, 영화를 만들고, 게임 캐릭터를 디자인하고, 가상 현실 콘텐츠를 제작하는 등 다양한 분야에서 활용됩니다.
* **마케팅 및 광고:**  맞춤형 광고를 생성하고, 제품 디자인을 개선하고, 고객 경험을 향상시키는 데 활용됩니다.
* **교육:**  개인 맞춤형 학습 자료와 게임을 제작하고, 학생들의 창의력을 키우는 데 활용됩니다.
* **의료:**  의료 영상을 분석하고, 새로운 의약품을 개발하고, 환자 맞춤형 치료 계획을 수립하는 데 활용됩니다.

### 결론

생성 AI는 다양한 분야에서 혁신적인 변화를 가져올 잠재력이 있습니다. 하지만, 윤리적 문제, 데이터 편향, 오용 가능성 등을 고려하여 책임감 있게 사용해야 합니다. 생성 AI의 장점과 단점을 잘 이해하고, 윤리적 기준을 명확히 설정하여 앞으로 나아가야 합니다. 


Time : 4.551941260000149 seconds


